Requires Data from the folder shared to Jignesh Sisodia Sir. If required we Need to change the line number 33 of the first cell to the data folder 

# **Imports**

In [ ]:
!pip install tensorflow==2.3.0
import os
import shutil
import gc
import copy
import sys
import cv2
import numpy as np
import pylab as pl
import tensorflow as tf
from time import sleep
from PIL import Image
from xml.etree import ElementTree
from IPython.display import clear_output
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import recall_score, classification_report
from tensorflow.python.keras.layers import Input, Dense, ConvLSTM2D
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Data/BlackBook/Caviar-Dataset')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Loading Train data**

In [ ]:
actions = ['Walk','Browse','Fight','LeftBag','Groups_Meeting','Rest']
X = None
y = None
for action in actions:
  with open(action+"/X.npy",'rb') as f:
    temp_X = np.load(f,allow_pickle=True)
    j = len(temp_X)%10
    if X is None:
      X = temp_X[:0-j]
    else:
      X = np.concatenate((X,temp_X[:0-j]),axis=0)
  with open(action+"/y.npy",'rb') as f:
    temp_y = np.load(f,allow_pickle=True)
    j = len(temp_y)%10
    if y is None:
      y = temp_y[:0-j]
    else:
      y = np.concatenate((y,temp_y[:0-j]),axis=0)

In [ ]:
print(X.shape)
print(y.shape)

(3850, 288, 384, 3)
(3850, 1)


In [ ]:
X = X[:,10:285,30:335]
X = np.array([cv2.resize(i, (224,224)) for i in X[:,:]])

In [ ]:
y[y == 1] = 0
y[y == 2] = 1

In [ ]:
X = X.reshape(385, 10 ,224, 224, 3)
y = y.reshape(385, 10, 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2)

In [ ]:
print(X_train.shape)

(308, 10, 224, 224, 3)


In [ ]:
video = Input(shape=(10, 224, 224, 3))
cnn_base = ResNet50(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
cnn_out = GlobalAveragePooling2D()(cnn_base.output)
cnn = Model(inputs=cnn_base.input, outputs=cnn_out)
cnn.trainable = False
x = TimeDistributed(cnn)(video)
x = GRU(512, return_sequences=True)(x) # Gated Recurrent Unit
x = Dropout(0.2)(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model([video], x)
model.summary()

earlystop = EarlyStopping(patience=7)
callbacks = [earlystop]

model.compile(optimizer='adam', loss='binary_crossentropy',metrics=["accuracy"], )
model.fit(X_train, y_train, batch_size=64, epochs=50, shuffle=False, callbacks=callbacks)

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10, 224, 224, 3)] 0         
_________________________________________________________________
time_distributed (TimeDistri (None, 10, 2048)          23587712  
_________________________________________________________________
gru (GRU)                    (None, 10, 512)           3933696   
_________________________________________________________________
dropout (Dropout)            (None, 10, 512)           0         
_________________________________________________________________
dense (Dense)                (None, 10, 1024)          525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 1024)          0         
_________________________________________________________________
dense_1 (Dense)              (None, 10, 1)            

KeyboardInterrupt: ignored

In [ ]:
model.save('model_resnet_gru_10fps_browse0_3.h5', overwrite=True)

In [ ]:
model = load_model('model_resnet_gru_10fps_browse0_3.h5')

In [ ]:
model.evaluate(X_test, y_test, batch_size=16)

In [ ]:
video = Input(shape=(10, 224, 224, 3))
cnn_base = ResNet50(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
cnn_out = GlobalAveragePooling2D()(cnn_base.output)
cnn = Model(inputs=cnn_base.input, outputs=cnn_out)
cnn.trainable = False
x = TimeDistributed(cnn)(video)
x = LSTM(512, return_sequences=True)(x) # Gated Recurrent Unit
x = Dropout(0.2)(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model([video], x)
model.summary()

earlystop = EarlyStopping(patience=7)
callbacks = [earlystop]

model.compile(optimizer='adam', loss='binary_crossentropy',metrics=["accuracy"], )
model.fit(X_train, y_train, batch_size=64, epochs=50, shuffle=False, callbacks=callbacks)

In [ ]:
model.save('model_resnet_lstm_10fps_browse0_3.h5', overwrite=True)

In [ ]:
model = load_model('model_resnet_lstm_10fps_browse0_3.h5')

In [ ]:
model.evaluate(X_test, y_test, batch_size=16)

In [ ]:
video = Input(shape=(10, 224, 224, 3))
cnn_base = ResNet50(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
cnn_out = GlobalAveragePooling2D()(cnn_base.output)
cnn = Model(inputs=cnn_base.input, outputs=cnn_out)
cnn.trainable = False
x = TimeDistributed(cnn)(video)
x = SimpleRNN(512, return_sequences=True)(x) # Gated Recurrent Unit
x = Dropout(0.2)(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model([video], x)
model.summary()

earlystop = EarlyStopping(patience=7)
callbacks = [earlystop]

model.compile(optimizer='adam', loss='binary_crossentropy',metrics=["accuracy"], )
model.fit(X_train, y_train, batch_size=64, epochs=50, shuffle=False, callbacks=callbacks)

In [ ]:
model.evaluate(X_test, y_test, batch_size=16)

In [ ]:
earlystop = EarlyStopping(patience=7)
callbacks = [earlystop]